In [212]:
%pip install pfapack

Note: you may need to restart the kernel to use updated packages.


In [213]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import block_diag
from pfapack import pfaffian
import math

In [214]:
def create_u_std_matrix(N1, N2, bc1, bc2):
    N = N1 * N2 # N1 为 a_1 方向 unit cell 数量
    M = np.zeros((2 * N, 2 * N)) # M 储存 u_jk(j 属于 A 子格，k 属于 B 子格)构型
    u = np.zeros((2 * N, 2 * N))
    
    if bc1 == "PBC":
        sign1 = 1
    elif bc1 == "APBC":
        sign1 = -1
    if bc2 == "PBC":
        sign2 = 1
    elif bc2 == "APBC":
        sign2 = -1    
    
    def index(n1, n2, sub):
        return n1 + n2 * N1 + N1 * N2 * sub # A子格对应 sub=0,B子格对应 sub=1
    
    for n2 in range(N2):
        for n1 in range(N1):
            j = index(n1, n2, 0) # j 为(n1, n2)unit cell 内 A 子格格点的单指标
            
            kx = index(n1, n2, 1) # kx 为(n1, n2)unit cell 内 B 子格格点的单指标
            M[j, kx] = 1 # j 与 kx 由 x-bond 相连，一定不跨越边界
            
            ky = index((n1 + 1) % N1, n2, 1) # ky 为 (n1, n2) 往 a1 方向走一步得到的 unit cell 内 B 子格格点的单指标j 与 ky 由 y-bond 相连
            if n1 == N1 - 1:
                M[j, ky] = sign1
            else:
                M[j, ky] = 1

            kz = index(n1, (n2 + 1) % N2, 1) # kz 为 (n1, n2) 往 a2 方向走一步得到的 unit cell 内 B 子格格点的单指标，j 与 kz 由 z-bond 相连
            if n2 == N2 - 1:
                M[j, kz] = sign2
            else:
                M[j, kz] = 1
      
    u = M - M.T                        
    return u

In [215]:
def create_t_matrix(N1, N2, Kx, Ky, Kz, u):
    N = N1 * N2
    t = np.zeros((N, N))
    K = np.array([Kx, Ky, Kz])
    for i in range(N):
        for j in range(N):
            sum = 0
            r_i_vec = np.array([i % N1, i // N1])
            r_j_vec = np.array([j % N1, j // N1])
            
            for delta_vec in np.array([[0, 0], [1, 0], [0, 1]]):
                tmp1 = (r_j_vec[0] - delta_vec[0] + N1) % N1
                tmp2 = (r_j_vec[1] - delta_vec[1] + N2) % N2
                if (np.array_equal(r_i_vec, np.array([tmp1, tmp2]))):
                    delta_func = 1
                else:
                    delta_func = 0
                    
                tmp3 = delta_vec[0] + delta_vec[1] * 2
                K_delta = K[tmp3]

                sum += delta_func * K_delta
                
            sum *= u[i, j + N]
            t[i, j] = sum
    return t

In [216]:
def create_t_plus_matrix(N1, N2, u):
    N = N1 * N2
    t_plus = np.zeros((N, N))
    def index1(UC_vec, sub):
        return N1 * UC_vec[1] + UC_vec[0] + sub * N
    list1 = [np.array([1, 0]), np.array([0, 1]), np.array([0, 0])]
    list2 = [np.array([0, 1]), np.array([0, 0]), np.array([1, 0])]
    for i in range(N):
        for j in range(N):
            r_i_vec = np.array([i % N1, i // N1])
            r_j_vec = np.array([j % N1, j // N1])
            sum = 0
            
            for delta1_vec, delta2_vec in zip(list1, list2):
                tmp1 = (r_i_vec[0]-delta1_vec[0] + N1) % N1 # r_i - delta_1 的 a_1 分量，考虑了周期性边界条件
                tmp2 = (r_i_vec[1]-delta1_vec[1] + N2) % N2 # r_i - delta_1 的 a_2 分量，考虑了周期性边界条件
                tmp3 = (r_j_vec[0]-delta2_vec[0] + N1) % N1 # r_j - delta_2 的 a_1 分量，考虑了周期性边界条件
                tmp4 = (r_j_vec[1]-delta2_vec[1] + N2) % N2 # r_j - delta_2 的 a_2 分量，考虑了周期性边界条件
                if (np.array_equal(np.array([tmp1, tmp2]) , np.array([tmp3, tmp4]))):
                    delta_func = 1
                else:
                    delta_func = 0
                sum += delta_func * u[index1(r_i_vec, 1), index1(np.array([tmp1, tmp2]), 0) ] * u[index1(r_j_vec, 1), index1(np.array([tmp3, tmp4]), 0)]

            t_plus[i][j] = sum
            
    return t_plus

In [217]:
def create_t_minus_matrix(N1, N2, u):
    N = N1 * N2
    t_minus = np.zeros((N, N))
    def index1(UC_vec, sub):
        return N1 * UC_vec[1] + UC_vec[0] + sub * N
    list1 = [np.array([-1, 0]), np.array([0, -1]), np.array([0, 0])]
    list2 = [np.array([0, -1]), np.array([0, 0]), np.array([-1, 0])]
    for i in range(N):
        for j in range(N):
            r_i_vec = np.array([i % N1, i // N1])
            r_j_vec = np.array([j % N1, j // N1])
            sum = 0
            
            for delta1_vec, delta2_vec in zip(list1, list2):
                tmp1 = (r_i_vec[0]-delta1_vec[0] ) % N1 # r_i - delta_1 的 a_1 分量，考虑了周期性边界条件
                tmp2 = (r_i_vec[1]-delta1_vec[1] ) % N2 # r_i - delta_1 的 a_2 分量，考虑了周期性边界条件
                tmp3 = (r_j_vec[0]-delta2_vec[0] ) % N1 # r_j - delta_2 的 a_1 分量，考虑了周期性边界条件
                tmp4 = (r_j_vec[1]-delta2_vec[1] ) % N2 # r_j - delta_2 的 a_2 分量，考虑了周期性边界条件
                if (np.array_equal(np.array([tmp1, tmp2]) , np.array([tmp3, tmp4]))):
                    delta_func = 1
                else:
                    delta_func = 0
                    
                sum += delta_func * u[index1(r_i_vec, 0), index1(np.array([tmp1, tmp2]), 1) ] * u[index1(r_j_vec, 0), index1(np.array([tmp3, tmp4]), 1)]

            t_minus[i][j] = sum
            
    return t_minus

In [218]:
def create_P_times_P_prime_matrix(N):
    P = np.zeros((2 * N, 2* N))
    for i in range(N):
        P[i, 2 * i] = 1
        P[i + N, 2 * i + 1] = 1
    # 计算 P_prime 矩阵
    I_N = np.eye(N) # N \times N 单位矩阵
    P_prime = np.kron(I_N, 0.5 * np.array([[1, 1j], [1, -1j]]))
    #print("P_prime:\n", P_prime)
    P_times_P_prime = np.dot(P, P_prime) # 矩阵乘法
    return P_times_P_prime

In [219]:
def add_vison_pair(N1, N2, n1, n2, sub, bond, u):
    def index(n1, n2, sub):
        return n1 + n2 * N1 + N1 * N2 * sub # A子格对应 sub=0,B子格对应 sub=1
    j = index(n1, n2, sub)
    if sub == 0:
        if bond == 1: # x-bond
            k = index(n1, n2, 1)
        elif bond == 2: # y-bond
            k = index((n1+1)%N1, n2, 1)
        elif bond == 3: # z-bond
            k = index(n1, (n2+1)%N2, 1)
    if sub == 1:
        if bond == 1: # x-bond
            k = index((n1-1+N1)%N1 , n2, 0)
        elif bond == 2: # y-bond
            k = index(n1, n2, 0)
        elif bond == 3: # z-bond
            k = index(n1, (n2-1+N2)%N2, 0)       

    u[j, k] = -u[j, k]
    u[k, j] = -u[k, j]
    return u
    

In [220]:
N1, N2, Kx, Ky, Kz, kappa = 10, 10, 1, 1, 1, 0.1
h_mag = np.array([0.1, 0.1, 0.1]) # 磁场
N = N1 * N2
n1, n2 = N1//2, N2//2 


In [221]:
results_H0 = {} # 空字典，以 (bc1, bc2) 为键，每个键储存对应边界条件下 H0 的本征值、本征矢量、基态能量
results_H_total = {} # 空字典，以 (bc1, bc2) 为键，每个键储存对应边界条件下 H_total = H0 + H_kappa 的本征值、本征矢量、基态能量
P_times_P_prime = create_P_times_P_prime_matrix(N)
for bc1 in ["PBC", "APBC"]:
    for bc2 in ["PBC", "APBC"]:
        if bc1 == "PBC" or bc2 == "PBC":
            continue
        u_std = create_u_std_matrix(N1, N2, bc1, bc2)
        t_std = create_t_matrix(N1, N2, Kx, Ky, Kz, u_std)
        Xi_std = t_std + t_std.T
        Delta_std = -(t_std - t_std.T)
        H_std = np.block([
        [Xi_std, Delta_std],
        [-Delta_std, -Xi_std.T]
        ])
        eigvals_std, eigvecs_std = np.linalg.eigh(H_std)
        print("GSenergy:", np.sum(eigvals_std[:N])/2)
        ################# A center #################
        hopping = [0,]
        u_configs = []
        eigvecs_configs = []
        U_tilde_configs = []
        U_configs = []
        W_tilde_configs = [0,]
        V_tilde_configs = [0,]
        F_tilde_configs = [0,]
        #M_configs = []
        
        
        #print('u_std:\n', u_std)
        u_configs.append(u_std)
        eigvecs_configs.append(eigvecs_std)
        u = eigvecs_std[:N, N:]
        v = eigvecs_std[N:, N:]
        U = np.block([[u, v.conj()], [v, u.conj()]])
        U_configs.append(U)
        U_tilde_configs.append(U.conj().T) # U(0)^\dagger
        
        I_N = np.eye(N, dtype=complex)
        U_0_prime = np.block([[I_N, I_N], [-1j*I_N, 1j*I_N]]) @ U
        U_0_prime_11 = U_0_prime[:N, :N]
        U_0_prime_12 = U_0_prime[:N, N:]
        U_0_prime_21 = U_0_prime[N:, :N]
        U_0_prime_22 = U_0_prime[:N, :N]
        
        for chi_bond in [1, 2, 3]:
            u_tmp = add_vison_pair(N1, N2, n1, n2, 0, chi_bond, u_std.copy())
            u_configs.append(u_tmp)
            
            t_tmp = create_t_matrix(N1, N2, Kx, Ky, Kz, u_tmp)
            Xi_tmp = t_tmp + t_tmp.T
            Delta_tmp = -(t_tmp - t_tmp.T)
            H_tmp = np.block([
            [Xi_tmp, Delta_tmp],
            [-Delta_tmp, -Xi_tmp.T]
            ])
            eigvals_tmp, eigvecs_tmp = np.linalg.eigh(H_tmp)
            print("energy:", np.sum(eigvals_tmp[:N])/2)
            #print(f"+:{eigvals_tmp[N:]}")
            #print(f"Delta:{np.sum(eigvals_tmp[:N])/2-np.sum(eigvals_std[:N])/2}")
            u_tmp = eigvecs_tmp[:N, N:]
            v_tmp = eigvecs_tmp[N:, N:]
            U_tmp = np.block([[u_tmp, v_tmp.conj()], [v_tmp, u_tmp.conj()]])
            print(f"center:A, bond={chi_bond}, U_tmp:\n{U_tmp}")
            eigvecs_configs.append(eigvecs_tmp)
            U_configs.append(U_tmp)
            U_tilde = U_tilde_configs[0] @ U_tmp
            U_tilde_configs.append(U_tilde)
            W_tilde = U_tilde[:N, :N]
            V_tilde = U_tilde[N:, :N]
            W_tilde_configs.append(W_tilde )
            V_tilde_configs.append(V_tilde)
            F_tilde_origin = V_tilde.conj() @ np.linalg.inv(W_tilde.conj())
            #print(f"F:{F_tilde_origin+F_tilde_origin.T}\n")
            #F_tilde = (F_tilde_origin - F_tilde_origin.T) / 2
            F_tilde_configs.append(F_tilde_origin)

            all_bonds = {1, 2, 3}
        for bond_1, bond_2 in [[1, 2], [2, 3], [3, 1]]: #bond_1=1,bond_2=2对应 x-y-bond跃迁
            bond_left = (all_bonds - {bond_1, bond_2}).pop() #比如 bond_1=2,bond_2=3,则bond_left=1
            i = N1 * n2 + n1 #i对应r\in A的unit cell单指标
            if bond_left == 1:
                j = N1 * n2 + n1
            elif bond_left == 2:
                j = N1 * n2 + (n1 + 1) % N1
            elif bond_left == 3:
                j = N1 * ((n2+1) % N2) + n1
            N_1_tilde = np.sqrt(np.abs(np.linalg.det(W_tilde_configs[bond_1])))
            N_2_tilde = np.sqrt(np.abs(np.linalg.det(W_tilde_configs[bond_2])))
            M = np.block([ [F_tilde_configs[bond_2], -np.eye(N)], [np.eye(N), -F_tilde_configs[bond_1].conj()] ])
            overlap =N_1_tilde * N_2_tilde * (-1)**(N*(N+1)/2) * pfaffian.pfaffian(M)
            #print("Pf(M):", pfaffian.pfaffian(M))
            #print("N_1:", N_1_tilde)
            #print("overlap:", overlap)
            middle_matrix = -np.linalg.inv(M) + np.block([ [np.zeros((N, N)), np.eye(N)], [np.zeros((N, N)), np.zeros((N, N))] ])
            left_WV = np.hstack([V_tilde_configs[bond_1].T.conj(), W_tilde_configs[bond_1].T.conj()])
            right_WV = np.vstack([W_tilde_configs[bond_2], V_tilde_configs[bond_2]])
            cor_alpha_1_alpha_2_dag = left_WV @ middle_matrix @ right_WV
            cor_alpha_1_c_i_A = left_WV @ middle_matrix @ np.vstack([U_0_prime_11[i, :].conj().reshape(-1, 1) ,U_0_prime_12[i, :].conj().reshape(-1, 1)])
            cor_alpha_1_c_j_B = left_WV @ middle_matrix @ np.vstack([U_0_prime_21[j, :].conj().reshape(-1, 1) ,U_0_prime_22[j, :].conj().reshape(-1, 1)])
            cor_c_i_A_alpha_2_dag = np.hstack([U_0_prime_12[i,:].reshape(1,-1), U_0_prime_11[i,:].reshape(1,-1) ]) @ middle_matrix @ right_WV
            cor_c_j_B_alpha_2_dag = np.hstack([U_0_prime_22[j,:].reshape(1,-1), U_0_prime_21[j,:].reshape(1,-1) ]) @ middle_matrix @ right_WV
            cor_c_i_A_c_j_B = np.hstack([U_0_prime_12[i,:].reshape(1,-1), U_0_prime_11[i,:].reshape(1,-1) ]) @ middle_matrix @ np.vstack(
                [U_0_prime_21[j, :].conj().reshape(-1, 1) ,U_0_prime_22[j, :].conj().reshape(-1, 1)]
            ) # 还是个1\times 1二维数组
            
            cor_alpha_1_c_i_A_c_j_B_alpha_2_dag = cor_alpha_1_c_i_A @ cor_c_j_B_alpha_2_dag - cor_alpha_1_c_j_B @ cor_c_i_A_alpha_2_dag + \
            cor_alpha_1_alpha_2_dag * cor_c_i_A_c_j_B[0, 0]

            hopping_amp = h_mag[bond_left-1] * overlap * (1j * cor_alpha_1_alpha_2_dag + cor_alpha_1_c_i_A_c_j_B_alpha_2_dag)
            print(f"{bc1,bc2},hopping:\n{bond_1}-{bond_2},amp:{hopping_amp}\n")
            hopping.append(hopping_amp)
        ################# B center #################
        



GSenergy: -157.44132763536774
energy: -157.21007743148215
center:A, bond=1, U_tmp:
[[-1.38066239e-16 -1.20826982e-01  9.56461010e-17 ... -4.89135101e-17
  -9.30986267e-17  1.45817659e-02]
 [-5.29670560e-17  4.88401160e-02 -2.10026289e-04 ... -2.08166817e-17
   5.02763169e-03  1.38676733e-02]
 [ 2.53279582e-16  6.88517067e-02 -7.40519064e-03 ...  1.38777878e-17
   9.45037685e-03  1.17825105e-02]
 ...
 [ 1.34499702e-01 -7.60596650e-03  4.26900759e-02 ...  8.31253876e-02
  -5.88809529e-02 -1.31904496e-01]
 [-1.14412281e-01 -3.01385331e-03  8.24496127e-02 ...  4.37016024e-02
  -1.04078412e-01 -1.11997804e-01]
 [ 1.70694818e-15  1.07644626e-02 -1.32171588e-01 ...  2.51534904e-16
  -1.39412102e-01 -8.13228377e-02]]
energy: -157.16129921511273
center:A, bond=2, U_tmp:
[[-0.02282357  0.02068448 -0.04669277 ... -0.00054195 -0.00254877
  -0.0145701 ]
 [-0.02282357 -0.02068448 -0.04669277 ... -0.00054195  0.00254877
  -0.0145701 ]
 [ 0.03715962 -0.00078086  0.06206661 ... -0.00026348  0.00733393
